In [1]:
import pandas as pd
import numpy as np
from google.colab import files
#upload from your local machine
uploaded = files.upload()
file_name = next(iter(uploaded))  # Gets the first uploaded filename
print(f"Uploaded: {file_name}")

Saving heart_failure_clinical_records_dataset.csv to heart_failure_clinical_records_dataset.csv
Uploaded: heart_failure_clinical_records_dataset.csv


In [2]:
# Load the dataset
df = pd.read_csv(file_name)
print(f"Shape: {df.shape}")
df.head()


Shape: (299, 13)


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [3]:
# Basic info
print("=== Data Types ===")
print(df.dtypes)

print("\n=== Missing Values ===")
print(df.isnull().sum())

# Quick stats
df.describe(include='all')

=== Data Types ===
age                         float64
anaemia                       int64
creatinine_phosphokinase      int64
diabetes                      int64
ejection_fraction             int64
high_blood_pressure           int64
platelets                   float64
serum_creatinine            float64
serum_sodium                  int64
sex                           int64
smoking                       int64
time                          int64
DEATH_EVENT                   int64
dtype: object

=== Missing Values ===
age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
time                        0
DEATH_EVENT                 0
dtype: int64


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [5]:
# Handling outliers in creatinine_phosphokinase
q1 = df['creatinine_phosphokinase'].quantile(0.25)
q3 = df['creatinine_phosphokinase'].quantile(0.75)
iqr = q3 - q1
upper_bound = q3 + 1.5 * iqr

df['creatinine_phosphokinase'] = df['creatinine_phosphokinase'].clip(upper=upper_bound)
print(f"\nClipped creatinine_phosphokinase values above {upper_bound}")


Clipped creatinine_phosphokinase values above 1280.25


In [7]:
# 2. Handle extreme serum_creatinine values (max was 9.4 which is medically unlikely)
df['serum_creatinine'] = df['serum_creatinine'].clip(upper=4.0)
print("Clipped serum_creatinine values above 4.0 mg/dL")

Clipped serum_creatinine values above 4.0 mg/dL


In [9]:
# Convert binary columns to boolean for clarity
binary_cols = ['anaemia', 'diabetes', 'high_blood_pressure', 'sex', 'smoking', 'DEATH_EVENT']
df[binary_cols] = df[binary_cols].astype(bool)

In [12]:
# Create age categories
bins = [40, 50, 60, 70, 80, 100]
labels = ['40-49', '50-59', '60-69', '70-79', '80+']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels)

In [14]:
# Verify changes
print("\n=== After Cleaning ===")
print(f"New columns: {df.columns.tolist()}")
print("\nData types after conversion:")
print(df.dtypes)

print("\n=== Updated Summary ===")
print(df[['creatinine_phosphokinase', 'serum_creatinine']].describe())


=== After Cleaning ===
New columns: ['age', 'anaemia', 'creatinine_phosphokinase', 'diabetes', 'ejection_fraction', 'high_blood_pressure', 'platelets', 'serum_creatinine', 'serum_sodium', 'sex', 'smoking', 'time', 'DEATH_EVENT', 'age_group']

Data types after conversion:
age                          float64
anaemia                         bool
creatinine_phosphokinase     float64
diabetes                        bool
ejection_fraction              int64
high_blood_pressure             bool
platelets                    float64
serum_creatinine             float64
serum_sodium                   int64
sex                             bool
smoking                         bool
time                           int64
DEATH_EVENT                     bool
age_group                   category
dtype: object

=== Updated Summary ===
       creatinine_phosphokinase  serum_creatinine
count                299.000000        299.000000
mean                 424.214883          1.332007
std                 

In [16]:
# Save cleaned data
df.to_csv('cleaned_heart_failure_data.csv', index=False)

In [17]:
# Save as CSV
output_file = 'cleaned_data.csv'
df.to_csv(output_file, index=False)

# Download to local machine
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>